In [1]:
import pandas as pd

import OpenDartReader
from pykrx import stock

In [2]:
pd.set_option('display.max.rows', 500)

In [3]:
api_key = '92c176817e681dcc4ad263eb3fa5182792b0b7a3'
dart = OpenDartReader(api_key)

In [4]:
# corp = '005930' # 기업코드
# bsns_year = '2020' # 기준연도
# fs_div = 'CFS' # 연결재무제표: CFS / 개별재무제표 :OFS
# reprt_code = '11011' # 사업보고서: 11011 / 반기보고서: 11012 / 1분기보고서: 11013 /  3분기보고서: 11014 

In [5]:
def get_equity(corp, bsns_year, fs_div, reprt_code) :
    df = dart.finstate_all(corp=corp, bsns_year=bsns_year, fs_div=fs_div, reprt_code=reprt_code)
    equity = df.loc[df['sj_div'].isin(['BS']) & df['account_id'].isin(['ifrs-full_Equity']), 'thstrm_amount']
    # 재무상태표: 'sj_div'='BS' / 자본총계 : 'account_id'='ifrs-full_Equity' / 값 : 'thstrm_amount'
    equity = int(equity.iat[0])
    # df의 값 추출
    return equity

In [6]:
def get_liability(corp, bsns_year, fs_div, reprt_code) :
    df = dart.finstate_all(corp=corp, bsns_year=bsns_year, fs_div=fs_div, reprt_code=reprt_code)
    liability = df.loc[df['sj_div'].isin(['BS']) & df['account_id'].isin(['ifrs-full_Liabilities']), 'thstrm_amount']
    # 재무상태표: 'sj_div'='BS' / 부채총계 : 'account_id'='ifrs-full_Liabilities' / 값 : 'thstrm_amount'
    liability = int(liability.iat[0])
    # df의 값 추출
    return liability

In [7]:
def get_profit_loss(corp, bsns_year, fs_div, reprt_code) :
    df = dart.finstate_all(corp=corp, bsns_year=bsns_year, fs_div=fs_div, reprt_code=reprt_code)
    profit_loss = df.loc[df['sj_div'].isin(['IS']) & df['account_id'].isin(['ifrs-full_ProfitLossAttributableToOwnersOfParent']), 'thstrm_amount']
    # 손익계산서: 'sj_div'='IS' / 당기순이익 : 'account_id'='ifrs-full_Equity' / 값 : 'thstrm_amount'
    profit_loss = int(profit_loss.iat[0])
    # df의 값 추출
    return profit_loss

In [8]:
def get_grossprofit(corp, bsns_year, fs_div, reprt_code) :
    df = dart.finstate_all(corp=corp, bsns_year=bsns_year, fs_div=fs_div, reprt_code=reprt_code)
    grossprofit = df.loc[df['sj_div'].isin(['IS']) & df['account_id'].isin(['ifrs-full_GrossProfit']), 'thstrm_amount']
    # 손익계산서: 'sj_div'='IS' / 매출총이익 : 'account_id'='ifrs-full_Equity' / 값 : 'thstrm_amount'
    grossprofit = int(grossprofit.iat[0])
    # df의 값 추출
    return grossprofit

In [9]:
def get_operating_profit(corp, bsns_year, fs_div, reprt_code) :
    df = dart.finstate_all(corp=corp, bsns_year=bsns_year, fs_div=fs_div, reprt_code=reprt_code)
    operating_profit=int(df.loc[df['account_nm'].str.contains('영업이익'), 'thstrm_amount'].iat[0])
    return operating_profit

In [10]:
df = stock.get_market_cap_by_ticker("20200625")
df.head()

,종가,시가총액,거래량,거래대금,상장주식수
티커,,,,,
005930,51900,309831714345000,18541624,970494909540,5969782550
000660,84300,61370599369500,3397112,287640892200,728002365
207940,815000,53924475000000,163339,133594217000,66165000
035420,269500,44268984952500,1196267,325381436500,164263395
068270,316000,42640845660000,918369,291688043500,134939385


In [11]:
df = stock.get_market_fundamental_by_ticker("20210104", market="KOSDAQ")
df.head()

,BPS,PER,PBR,EPS,DIV,DPS
티커,,,,,,
060310,745,0.000000,3.029297,0,0.000000,0
054620,10530,0.000000,0.759766,0,0.000000,0
265520,7468,38.000000,3.410156,671,0.199951,50
211270,5479,163.375000,1.519531,51,0.839844,70
035760,134364,26.703125,1.059570,5310,0.990234,1400


In [12]:
tickers = stock.get_market_ticker_list("20190225", market="KOSPI")  # KOSPI / KOSDAQ / KONEX / ALL / default = KOSPI
print(tickers[:10]) 

['095570', '068400', '006840', '027410', '282330', '138930', '001460', '001465', '001040', '079160']


In [13]:
tickers = stock.get_market_ticker_list("20190225", market="KOSPI")  # KOSPI / KOSDAQ / KONEX / ALL / default = KOSPI

temp_dic = {'tickers':[], 'quality':[]}
for ticker in tickers :
    try:
        corp = ticker # 기업코드
        bsns_year = '2020' # 기준연도
        fs_div = 'CFS' # 연결재무제표: CFS / 개별재무제표 :OFS
        reprt_code = '11011' # 사업보고서: 11011 / 반기보고서: 11012 / 1분기보고서: 11013 /  3분기보고서: 11014 
        operating_profit = get_operating_profit(corp, bsns_year, fs_div, reprt_code)
        equity = get_equity(corp, bsns_year, fs_div, reprt_code)
        liability = get_liability(corp, bsns_year, fs_div, reprt_code)
        quality = operating_profit / (equity + liability)
        temp_dic['tickers'].append(ticker)
        temp_dic['quality'].append(quality)
    except :
        pass

In [14]:
quality=pd.DataFrame(temp_dic)
quality['quality_rank'] = quality['quality'].rank(ascending=False)
quality.sort_values(by='quality_rank').head(10)

,tickers,quality,quality_rank
526,042700,0.203842,1.0
12,000990,0.203071,2.0
365,036570,0.202095,3.0
469,021240,0.200209,4.0
472,284740,0.191826,5.0
28,234080,0.180932,6.0
49,051900,0.179502,7.0
279,003230,0.171402,8.0
113,005320,0.167600,9.0
297,248170,0.159329,10.0


In [15]:
value=stock.get_market_fundamental_by_ticker(date='20200529')
value=value.reset_index()
value=value.rename(columns={'티커':'ticker'})
value=value[['ticker','PBR']]
value=value[value['PBR']>0]
value['value_rank'] =value['PBR'].rank()
value.sort_values(by='value_rank').head(10)

,ticker,PBR,value_rank
821,088350,0.099976,1.0
888,005010,0.130005,2.0
465,003030,0.140015,3.5
467,058650,0.140015,3.5
146,012320,0.150024,5.0
443,200880,0.160034,6.0
853,004020,0.170044,7.0
290,082640,0.180054,8.5
880,298040,0.180054,8.5
617,003200,0.189941,12.0


In [16]:
quality_value = pd.merge(quality, value, how='inner', left_on='tickers', right_on='ticker')
quality_value['total_rank'] = quality_value['quality_rank'] + quality_value['value_rank']
quality_value.sort_values(by='total_rank')

,tickers,quality,quality_rank,ticker,PBR,value_rank,total_rank
105,013580,0.084433,63.0,013580,0.340088,111.5,174.5
185,001230,0.054387,144.0,001230,0.219971,30.5,174.5
22,012630,0.056635,128.0,012630,0.260010,51.5,179.5
532,004960,0.069160,95.0,004960,0.320068,95.5,190.5
347,002030,0.050892,162.0,002030,0.229980,35.5,197.5
...,...,...,...,...,...,...,...
455,109070,-0.151496,587.0,109070,4.058594,747.0,1334.0
452,012600,-0.227644,596.0,012600,3.730469,741.0,1337.0
450,011000,-0.135162,583.0,011000,11.796875,771.0,1354.0
514,123690,-0.225442,595.0,123690,6.421875,760.5,1355.5


In [17]:
quality_value['rank'] = quality_value['total_rank'].rank()
quality_value.sort_values(by='total_rank').head(10)

,tickers,quality,quality_rank,ticker,PBR,value_rank,total_rank,rank
105,013580,0.084433,63.0,013580,0.340088,111.5,174.5,1.5
185,001230,0.054387,144.0,001230,0.219971,30.5,174.5,1.5
22,012630,0.056635,128.0,012630,0.260010,51.5,179.5,3.0
532,004960,0.069160,95.0,004960,0.320068,95.5,190.5,4.0
347,002030,0.050892,162.0,002030,0.229980,35.5,197.5,5.0
37,001940,0.052846,155.0,001940,0.260010,51.5,206.5,6.0
10,000590,0.049120,169.0,000590,0.250000,44.0,213.0,7.0
170,015230,0.077479,74.0,015230,0.370117,139.5,213.5,8.0
70,036530,0.055973,134.0,036530,0.310059,87.0,221.0,9.0
560,004560,0.051506,159.0,004560,0.280029,63.5,222.5,10.0


In [18]:
p1=quality_value[quality_value['rank']<quality_value.quantile(q=.2, interpolation='nearest')['rank']]
p1=p1.sort_values(by='rank')
list(p1['tickers'].unique())

['001230',
 '013580',
 '012630',
 '004960',
 '002030',
 '001940',
 '000590',
 '015230',
 '036530',
 '004560',
 '002460',
 '084010',
 '294870',
 '009970',
 '013570',
 '003300',
 '002170',
 '002020',
 '032560',
 '092230',
 '072710',
 '007340',
 '003650',
 '004490',
 '012320',
 '300720',
 '000240',
 '006650',
 '183190',
 '161390',
 '025000',
 '005010',
 '003030',
 '002310',
 '001800',
 '200880',
 '009070',
 '025540',
 '014830',
 '000070',
 '002200',
 '057050',
 '210540',
 '084690',
 '060980',
 '029460',
 '089470',
 '100840',
 '004090',
 '123700',
 '103140',
 '053210',
 '004690',
 '155660',
 '016590',
 '003070',
 '007860',
 '161000',
 '058430',
 '001790',
 '042670',
 '093050',
 '015760',
 '145990',
 '034120',
 '023810',
 '001530',
 '006260',
 '003480',
 '090350',
 '282690',
 '006120',
 '267290',
 '000320',
 '018670',
 '003200',
 '005960',
 '003960',
 '267270',
 '000500',
 '078930',
 '003550',
 '298040',
 '092440',
 '001390',
 '025530',
 '005490',
 '009200',
 '000180',
 '005720',
 '014790',

In [20]:
p1.to_csv('C:/Users/ajcltm/Desktop/p1.csv')